In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://archive.ubuntu.com/ubuntu focal-updates/main amd6

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-11 03:13:24--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2023-03-11 03:13:24 (8.08 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()
# spark = SparkSession.builder.appName("BigData-HW-1").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [6]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://big-data-etl.s3.amazonaws.com/amazon_reviews_us_Sports_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv"), sep='\t')

# Show DataFrame
df.show()


In [8]:
# Get the number of rows in the DataFrame.
df.count()


4850360

# Transform the Data

## Create the "review_id_table".

In [46]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.

# review_id_df = df.select('review_id','customer_id','product_id','product_parent',to_date('review_date','yyyy-mm-dd'))
review_id_df = df.select('review_id','customer_id','product_id','product_parent',to_date(df.review_date).alias('review_date'))
review_id_df.show()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1WBPB8MDCCN8F|   48945260|B012P7UPSM|     409940130| 2015-08-31|
|R32M0YEWV77XG8|    5782091|B001GQ3VHG|     657746679| 2015-08-31|
| RR8V7WR27NXJ5|   45813853|B008VS8M58|     962442336| 2015-08-31|
|R1MHO5V9Z932AY|    1593730|B005F06F4U|      74305227| 2015-08-31|
|R16PD71086BD2V|   29605511|B010T4IE2C|     787185588| 2015-08-31|
|R1Z8IFGWTRWXT6|   11112959|B004RKJGLS|      94127483| 2015-08-31|
|R3AUMSHAW73HWN|     108031|B005V3DCBU|     526977496| 2015-08-31|
|R2KWDWFOHGX6FL|   13981540|B00MHT9WN8|      26521265| 2015-08-31|
|R3H9543FWBWFBU|   37993909|B001CSIRQ8|     652431165| 2015-08-31|
| RUANXOQ9W3OU5|   26040213|B001KZ3NOO|     635861713| 2015-08-31|
|R31673RTGEZSW7|   34657602|B00005RCQS|      72099763| 2015-08-31|
|R22OQLFSH42RCM|   14346192|B00FA7RWVI|     757354022| 2015-08

## Create the "products" Table

In [79]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
print("Original Dataframe count: ", df.count())
products_df = df.select('product_id','product_title').dropDuplicates(['product_id'])
print("New Dataframe count: ", products_df.count())

products_df.show()


Original Dataframe count:  4850360
New Dataframe count:  1046150
+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0000031852|Girls Ballet Tutu...|
|0000031909|Mystiqueshapes Gi...|
|0201593246|Official Arsenal ...|
|0395911737|Peterson Field Gu...|
|0531904822|Police Protect & ...|
|0607968699|Black Hills Natio...|
|0615154077|Let It Rain: The ...|
|0615203868|Fitness Republic ...|
|061520581X|     Jennys Heritage|
|0615247180|Coursing Around.....|
|0615541003|Advanced Mobility...|
|0615614604|Golf Fitness Over...|
|061586628X|Handee Band: Resi...|
|0641649975|Cardinal Professi...|
|0641649983|Aluminum Case Del...|
|0641869169|Contigo 14 oz Ell...|
|0679771611|Uncle Sam's Guide...|
|068143273X|Watchmen Who Watc...|
|0736060332|Power Systems Adv...|
|0736065768|Power Systems Exp...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [84]:
from pyspark.sql.functions import count
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
# customers_df = df.groupby('customer_id').agg({"customer_id":"count"}.alias("customer_count"))
customers_df = df.groupby('customer_id').agg(count("customer_id").alias("customer_count"))
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45193257|            21|
|   44934924|             1|
|   11614909|             1|
|   19186664|             1|
|   50492011|             1|
|   12988293|             3|
|   22848554|             4|
|   42126527|             1|
|   48782599|             1|
|   34146651|            10|
|   43194960|             3|
|   14127895|             1|
|   30636778|            44|
|   30274993|             1|
|   28069251|             1|
|   20545025|             5|
|   28029004|             4|
|   51290010|             1|
|   37367363|             1|
|    2583392|             7|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [66]:
from pyspark.sql.types import IntegerType
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
# vine_df = df.select('review_id', \
#                     col('star_rating').cast("int").as("star_rating"), \
#                     col('helpful_votes').cast("int").as("helpful_votes"), \
#                     col('total_vines').cast("int").as("total_vines"))

vine_df = df.select('review_id','star_rating','helpful_votes','total_votes','vine')
vine_df = vine_df.withColumn('star_rating', vine_df['star_rating'].cast(IntegerType()))
vine_df = vine_df.withColumn('helpful_votes', vine_df['helpful_votes'].cast(IntegerType()))
vine_df = vine_df.withColumn('total_votes', vine_df['total_votes'].cast(IntegerType()))

vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|
|R32M0YEWV77XG8|          5|            1|          1|   N|
| RR8V7WR27NXJ5|          1|            0|          0|   N|
|R1MHO5V9Z932AY|          5|            0|          0|   N|
|R16PD71086BD2V|          5|            0|          1|   N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|
|R3AUMSHAW73HWN|          4|            2|          3|   N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|
|R3H9543FWBWFBU|          5|            1|          1|   N|
| RUANXOQ9W3OU5|          5|            0|          0|   N|
|R31673RTGEZSW7|          5|            2|          2|   N|
|R22OQLFSH42RCM|          5|            1|          1|   N|
|R12LEL4F3TSZUJ|          5|            2|          2|   N|
|R2L9XWD03072NI|          5|            

# Load

In [35]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/<RDS Database>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [68]:
# Print all schemas
print("Review_ID Schema:")
print(review_id_df.printSchema())

print("Products")
print(products_df.printSchema())

print("Customers")
print(customers_df.printSchema())

print("Vine")
print(vine_df.printSchema())

Review_ID Schema:
root
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- review_date: date (nullable = true)

None
Products
root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)

None
Customers
root
 |-- customer_id: string (nullable = true)
 |-- count(customer_id): long (nullable = false)

None
Vine
root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)

None


In [69]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='sports_review', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='sports_products', mode=mode, properties=config)

In [87]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='sports_customers', mode=mode, properties=config)

In [88]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='sports_vine', mode=mode, properties=config)